In [1]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn.itemknn import ItemKNN
import pandas as pd
from src.evaluation import compute_mrr
from src.utils.sparse_matrix import interactions_to_sparse_matrix

In [2]:
dataset = Dataset()

In [5]:
full_data = dataset.get_train_sessions()

In [12]:
full_data["sess_id_copy"] = full_data[SESS_ID].copy()

In [16]:
full_data_dict = full_data.groupby(SESS_ID)["sess_id_copy",ITEM_ID].apply(list)

/tmp/ipykernel_69039/706950461.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_data_dict = full_data.groupby(SESS_ID)["sess_id_copy",ITEM_ID].apply(list)


In [18]:
full_data_dict.values[0]

['sess_id_copy', 'item_id']

In [11]:
full_data_dict[18421]

KeyError: 18421

In [102]:
unique_sess_ids=dataset.get_train_sessions()[SESS_ID].unique()

In [108]:
torch.unsqueeze(torch.tensor(unique_sess_ids, dtype=torch.int64), dim=0)

tensor([[      3,      13,      18,  ..., 4439994, 4439999, 4440001]])

In [3]:
split_dict = dataset.get_split()

In [4]:
train, train_label = split_dict[TRAIN]

In [90]:
import torch
a = train.groupby(SESS_ID)[ITEM_ID].apply(list)

In [96]:
a.to_dict()

{13: [13198],
 18: [15459, 2085, 3377],
 19: [21726,
  5336,
  21550,
  16911,
  6978,
  6967,
  3676,
  5635,
  7005,
  23259,
  7417,
  16789,
  23510,
  19985,
  10786,
  1470,
  15977],
 28: [9720, 9720, 14258, 18492],
 31: [21891, 13739, 1725, 23209, 22292, 1725, 3553, 216],
 36: [22359, 21441],
 42: [8756, 17332, 14743, 18989],
 44: [19194, 14419],
 48: [7073, 22250],
 49: [18026],
 52: [20777, 10148],
 75: [4541],
 77: [11132, 11132, 2667, 6684, 20514],
 113: [1806, 14530, 13271, 16667, 17906, 5205],
 115: [14028, 16621, 13463, 16621, 14028, 16621, 8798, 8798],
 119: [22039, 21834, 20514],
 140: [15797, 11724, 9588],
 151: [5134, 23364],
 154: [17849],
 156: [2904],
 170: [2785],
 181: [13848, 8626, 4049, 1701, 4049],
 182: [20069],
 184: [12110],
 188: [1902],
 195: [3428, 14023, 11570, 19942, 13551, 20782, 17740, 19152, 22606, 20782],
 207: [15128, 15128, 21160, 21160],
 208: [22129],
 226: [21979, 17120, 18420, 21979, 23401, 1958, 1958],
 227: [13161, 18379, 15626],
 235: [12

In [29]:
user_list = np.arange(1000)
filterd_train = train[train[SESS_ID].isin(user_list)]


In [31]:
filterd_train[[SESS_ID, ITEM_ID]].values

array([[   13, 13198],
       [   18, 15459],
       [   18,  2085],
       ...,
       [  997, 13874],
       [  997, 13305],
       [  997, 19884]])

In [24]:
import torch
torch.Tensor(dataset.get_oh_item_features().values).shape[1]

963

In [55]:
from src.utils.pandas_utils import remap_column_consecutive

In [77]:
mapped_df, _ = remap_column_consecutive(train, column_name=SESS_ID)

In [79]:
indices = mapped_df[[SESS_ID, ITEM_ID]].values
values = np.ones(len(indices))
unique_sess = mapped_df[SESS_ID].nunique()

In [80]:
torch.sparse_coo_tensor(torch.tensor(list(zip(*indices))), torch.tensor(values, dtype=torch.float32), [unique_sess, 23_691])

tensor(indices=tensor([[     0,      1,      1,  ..., 799999, 799999, 799999],
                       [ 13198,  15459,   2085,  ...,   5045,   7161,  13367]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(800000, 23691), nnz=3790413, layout=torch.sparse_coo)

In [54]:
indices

array([[   13, 13198],
       [   18, 15459],
       [   18,  2085],
       ...,
       [  997, 13874],
       [  997, 13305],
       [  997, 19884]])

In [53]:
torch.Tensor(indices)

tensor([[1.3000e+01, 1.3198e+04],
        [1.8000e+01, 1.5459e+04],
        [1.8000e+01, 2.0850e+03],
        ...,
        [9.9700e+02, 1.3874e+04],
        [9.9700e+02, 1.3305e+04],
        [9.9700e+02, 1.9884e+04]])

In [52]:
torch.sparse_coo_tensor(torch.Tensor(indices), torch.tensor(values, dtype=torch.float32), (unique_sess, 23691))

RuntimeError: number of dimensions must be sparse_dim (672) + dense_dim (0), but got 2

In [5]:
sparse_interaction, user_mapping_dict, _ = interactions_to_sparse_matrix(
            train,
            items_num=dataset._ITEMS_NUM,
            users_num=None,
        )

In [15]:
user, item = zip(*train[[SESS_ID, ITEM_ID]].values)

In [17]:
from collections import defaultdict
from tqdm import tqdm
user_dict = defaultdict(lambda: {})
for u, i in tqdm(zip(user, item)):
    if i not in user_dict[u]:
        user_dict[u][i] = 1

3790413it [00:02, 1301889.37it/s]


In [18]:
user_dict

defaultdict(<function __main__.<lambda>()>,
            {13: {13198: 1},
             18: {15459: 1, 2085: 1, 3377: 1},
             19: {21726: 1,
              5336: 1,
              21550: 1,
              16911: 1,
              6978: 1,
              6967: 1,
              3676: 1,
              5635: 1,
              7005: 1,
              23259: 1,
              7417: 1,
              16789: 1,
              23510: 1,
              19985: 1,
              10786: 1,
              1470: 1,
              15977: 1},
             28: {9720: 1, 14258: 1, 18492: 1},
             31: {21891: 1,
              13739: 1,
              1725: 1,
              23209: 1,
              22292: 1,
              3553: 1,
              216: 1},
             36: {22359: 1, 21441: 1},
             42: {8756: 1, 17332: 1, 14743: 1, 18989: 1},
             44: {19194: 1, 14419: 1},
             48: {7073: 1, 22250: 1},
             49: {18026: 1},
             52: {20777: 1, 10148: 1},
             75:

In [9]:
train[[SESS_ID, ITEM_ID]].values

array([[     13,   13198],
       [     18,   15459],
       [     18,    2085],
       ...,
       [4439999,    5045],
       [4439999,    7161],
       [4439999,   13367]])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
sim = cosine_similarity(sparse_interaction.T, dense_output=False)

In [8]:
sim

<23691x23691 sparse matrix of type '<class 'numpy.float32'>'
	with 17019887 stored elements in Compressed Sparse Row format>

In [12]:
len(sim.indices)

17019887

In [11]:
len(sim.indptr)

23692

In [19]:
import torch
a = torch.sparse_csr_tensor(sim.indptr, sim.indices, sim.data, dtype = torch.float32)

In [20]:
a.shape

torch.Size([23691, 23691])

In [24]:
import similaripy
similaripy.normalization.normalize(sim, norm="l1", axis=0)

<23691x23691 sparse matrix of type '<class 'numpy.float32'>'
	with 17019887 stored elements in Compressed Sparse Row format>

In [ ]:
(17019887)/23691**2

In [ ]:
import similaripy

In [ ]:

sim = similaripy.cosine(sparse_interaction.T, k=200)

In [ ]:
sim

### Try to construct graph with PyG

In [ ]:
from torch_geometric.data import Data
import torch
import numpy as np

In [ ]:
coo_sim = sim.tocoo()

In [ ]:
coo_sim.row

In [ ]:
coo_sim.col

In [ ]:
sim.data

In [ ]:
# we have to create the edges both ways
start_node_edge = np.concatenate([coo_sim.row, coo_sim.col])
arrival_node_edge = np.concatenate([coo_sim.col, coo_sim.row])

In [ ]:

# create the graph
edge_index = torch.tensor(np.array([start_node_edge, arrival_node_edge]), dtype=torch.long)

In [ ]:
item_features = dataset.get_oh_item_features()

In [ ]:
item_features

In [ ]:
x = torch.tensor(item_features.values, dtype=torch.float)

In [ ]:
x

In [ ]:
data = Data(x=x, edge_index=edge_index)

In [ ]:
data.num_nodes

In [ ]:
data.num_edges

In [ ]:
data.has_isolated_nodes()

In [ ]:
data.has_self_loops()

In [ ]:
data.is_directed()

In [ ]:
from torch_geometric.utils import to_networkx

In [ ]:
#nx_graph = to_networkx(data, to_undirected=True)

In [ ]:
#import networkx as nx
#[len(c) for c in sorted(nx.connected_components(nx_graph), key=len, reverse=True)]

In [ ]:
graph_dataset = data

In [ ]:
graph_dataset

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(graph_dataset.num_node_features, 256)
        self.conv2 = GCNConv(256, 128)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
model = GCN().to(device)
data = graph_dataset.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)    
out = model(data)
    
    

In [ ]:
out.shape